<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Domi-DEV/Productive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Choice, Fine-tuning and Evaluation**

After having set up the QA-dataset, we are now capable of evaluating different models on the task that the dataset implicitly represents.
For that, we have to create all corresponding functions that translates our dataset entries into model input and vice versa the model output to humanly understandable text.
Therafter - or at the same time, as we will do it - the evaluation of that created output has to happen.

On the basis of this data, we will decide which model we will fine-tune afterwards, to improve the model's performance even more.

At last, we will test the newly fine-tuned model against the other models evaluated before






But first things first, let's start with installing and importing all the necessary packages.

In [1]:
!pip install evaluate
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

In [35]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
import numpy as np
import urllib
from itertools import chain, combinations
from transformers import AutoTokenizer, AutoModelForMultipleChoice, AutoModelForQuestionAnswering, TrainingArguments, pipeline, Trainer, DataCollatorWithPadding, XLNetForMultipleChoice
import torch
import requests
import evaluate
import numpy as np
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from datasets import Dataset
from typing import Optional, Union
from dateutil import parser
from datetime import datetime
import os
import re
from google.colab import drive


## Preprocess dataset

Here we split the previously created QA-dataset into train and validation dataset. For that, we load it from the public github account, where it was uploaded before.
Additionnaly, we prepare the dataset to later be useful for response-generation and fine-tuning of a model

In [3]:
# Load datset
url = "https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/datasets/combined_qa_dataset.json"
data = pd.read_json(url)
# Convert to DataFrame for easy handling
df = pd.DataFrame(data)

# Map the intended answer to the index of the option
df['label'] = df.apply(lambda x: np.array([1 if option in x['intended_answer'] else 0 for option in x['options']]) if x['type'] in ['SINGLE_SELECT', 'MULTI_SELECT'] else np.array([0]), axis=1)
df['stratify_key'] = df['difficulty'] + '_' + df['type']

# Convert to Huggingface Dataset dataset
qa_dataset = Dataset.from_pandas(df)

In [4]:
# split dataset into train and validation (here called test) dataset with stratifying with question type and difficulty of context
qa_dataset = qa_dataset.class_encode_column(
    "stratify_key"
).train_test_split(test_size=0.2, stratify_by_column="stratify_key", seed=42).remove_columns("stratify_key")

Casting to class labels:   0%|          | 0/1381 [00:00<?, ? examples/s]

In [5]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'type', 'options', 'intended_answer', 'context', 'difficulty', 'label'],
        num_rows: 1104
    })
    test: Dataset({
        features: ['question', 'type', 'options', 'intended_answer', 'context', 'difficulty', 'label'],
        num_rows: 277
    })
})

In [6]:
# Label column: the intended_answer as list of binary variables for every option if question type is mc questions, list of entry 0 else
qa_dataset["train"]["label"][:5]

[[0, 1, 0, 0, 0, 0, 0, 0], [0, 1], [0, 1, 0, 1, 0], [0], [1, 0]]

### Generate model output

After the creation of the QA-dataset, it's time for generating model output for different Huggingface models.

In [7]:
def tokenize_function(example, tokenizer):
    '''
    Converts the string input, which is a question with its context and the given options for multi-/single-select questions, into IDs the model later can make sense of. Distinguishes between multi-/single-select and the other questions
    parameters:
    - expample: question of the QA-dataset with all its entries (question, context, options, type are urgently necessary)
    - tokenizer: tokenizer of the model
    output:
    - tokenized: tokenized input example
    '''
    if example["type"] == "SINGLE_SELECT" or example["type"] == "MULTI_SELECT":
      number_of_options = len(example["options"])
      first_sentence = [[example["context"]] * number_of_options]  # Repeat context for each option
      second_sentence = [[example["question"] + " " + option] for option in example["options"]]  # Pair with each option
      tokenized = tokenizer(
          sum(first_sentence, []),
          sum(second_sentence, []),
          padding="longest",
          truncation=True
      )
      # Un-flatten
      return {k: [v[i:i+number_of_options] for i in range(0, len(v), number_of_options)] for k, v in tokenized.items()}

    elif example['type'] == 'NUMBER':
      tokenized = tokenizer(
          example['context'],
          example['question'],
          truncation="only_second",
          max_length=384,
          padding="max_length",
          return_tensors="pt"
      )
    else:
      tokenized = tokenizer(
          example['question'],
          example['context'],
          truncation="only_second",
          max_length=384,
          padding="max_length",
          return_tensors="pt"
      )

    return tokenized

In [8]:
def model_output(mc_model, mc_tokenizer, oe_model, oe_tokenizer, questions, sum_pipeline=None, mc_metric=None, oe_metric=None):
    '''
    model_output -> creates output for every question in the dataset and safes it in a list of dicts. One dic has keys 'answer', 'predicted_answer', 'type'
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - questions: QA-dataset in json format
    '''
    answer_comparison = []
    mc_answer_comparison = []
    mc_model_name = mc_model.config._name_or_path
    oe_model_name = oe_model.config._name_or_path

    for index, question in questions.iterrows():
        context = question['context']
        question_text = question['question']
        options = question['options']
        question_type = question['type']
        difficulty = question['difficulty']

        mc_question_type = question_type in ["MULTI_SELECT", "SINGLE_SELECT"]

        if question_type == "MULTI_SELECT":
          intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer = multi_select_model_output(mc_model, mc_tokenizer, question, mc_metric)
        elif question_type == "SINGLE_SELECT":
          intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer = single_select_model_output(mc_model, mc_tokenizer, question, mc_metric)
        elif question_type == "TEXT":
          intended_answer, predicted_answer = text_model_output(question, sum_pipeline)
          continue
        elif question_type == "NUMBER":
          intended_answer, predicted_answer = number_model_output(oe_model, oe_tokenizer, question, oe_metric)
        elif question_type == "DATE":
          intended_answer, predicted_answer = date_model_output(oe_model, oe_tokenizer, question, oe_metric)
        else:
          continue
        if predicted_answer != intended_answer:
          print('======= Wrong answer =======')
          print(f"Question: {question_text}")
          print(f"Context: {context}")
          print(f"The intended answer was: {intended_answer}")
          print(f"The predicted answer was: {predicted_answer}")
          if mc_question_type:
            print(f"The intended answer in BINARY was: {intended_answer_binary}")
            print(f"The predicted answer in BINARY was: {predicted_answer_binary}\n")
          else:
            print("")
        if mc_question_type:
          mc_answer_comparison.append({'model': mc_model_name, 'intended_answer_binary': intended_answer_binary, 'predicted_answer_binary': predicted_answer_binary, 'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})
        else:
          answer_comparison.append({'model': oe_model_name, 'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})
    if mc_metric is not None:
      try:
        mc_metric_result = mc_metric.compute()
      except:
        mc_metric_result = None
    else:
      mc_metric_result = None
    if oe_metric is not None:
      try:
        oe_metric_result = oe_metric.compute()
      except:
        oe_metric_result = None
    else:
      oe_metric_result = None
    return mc_answer_comparison, answer_comparison, mc_metric_result, oe_metric_result


In [9]:
def map_model_output(mc_model, mc_tokenizer, oe_model, oe_tokenizer, question, sum_pipeline=None, mc_metric=None, oe_metric=None):
    '''
    model_output -> creates output for every question in the dataset and safes it in a list of dicts. One dic has keys 'answer', 'predicted_answer', 'type'
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - questions: QA-dataset in json format
    '''
    answer_comparison = []
    mc_answer_comparison = []
    context = question['context']
    question_text = question['question']
    options = question['options']
    question_type = question['type']
    difficulty = question['difficulty']

    mc_question_type = question_type in ["MULTI_SELECT", "SINGLE_SELECT"]

    if question_type == "MULTI_SELECT":
      intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer = multi_select_model_output(mc_model, mc_tokenizer, question, mc_metric)
    elif question_type == "SINGLE_SELECT":
      intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer = single_select_model_output(mc_model, mc_tokenizer, question, mc_metric)
    elif question_type == "TEXT":
      intended_answer, predicted_answer = text_model_output(question, sum_pipeline)
    elif question_type == "NUMBER":
      intended_answer, predicted_answer = number_model_output(oe_model, oe_tokenizer, question, oe_metric)
    elif question_type == "DATE":
      intended_answer, predicted_answer = date_model_output(oe_model, oe_tokenizer, question, oe_metric)
    else:
      return None, None
    if predicted_answer != intended_answer:
      print('======= Wrong answer =======')
      print(f"Question: {question_text}")
      print(f"Context: {context}")
      print(f"The intended answer was: {intended_answer}")
      print(f"The predicted answer was: {predicted_answer}")
      if mc_question_type:
        print(f"The intended answer in BINARY was: {intended_answer_binary}")
        print(f"The predicted answer in BINARY was: {predicted_answer_binary}\n")
      else:
        print("")
    if mc_question_type:
      mc_answer_comparison.append({'intended_answer_binary': intended_answer_binary, 'predicted_answer_binary': predicted_answer_binary, 'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})
    else:
      answer_comparison.append({'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})

    return mc_answer_comparison, answer_comparison


Single-select output

In [10]:
def single_select_model_output(model, tokenizer, question, metric=None):
    '''
    Handles a question, its context and its options for a single-select question and generates output
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the correct/intended answer as a list of a string
    - predicted_answer: the predicted answer as a list of a string
    '''
    intended_answer = question['intended_answer'][0]
    options = question['options']

    # creating input ids by tokenizing the question
    input_ids = tokenize_function(question, tokenizer)
    input_ids = {key: torch.tensor(array) for key, array in input_ids.items()}

    # generating the output
    outputs = model(**input_ids)
    logits = outputs.logits  # Shape: [batch_size, num_choices]
    print(logits)
    # Predict the option with the highest score
    predicted_option = torch.argmax(logits, dim=1).item()

    predicted_answer_binary = [0] * len(options)
    predicted_answer_binary[predicted_option] = 1

    intended_answer_binary = [1 if option == intended_answer else 0 for option in options]

    if metric is not None:
      metric.add_batch(predictions=predicted_answer_binary, references=intended_answer_binary)

    return intended_answer, intended_answer_binary, predicted_answer_binary, options[predicted_option]


Multi-select output

In [11]:
def multi_select_model_output(model, tokenizer, question, metric=None):
    '''
    Handles a question, its context and its options for a multi-select question and generates output as a list of indices of the predicted answers. Ticks every option whose probability is at least 90% of the best option (softmax)
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the correct/intended answers as a list of strings
    - predicted_answer: the predicted answers as a list of strings
    '''
    intended_answer = question['intended_answer']
    options = question['options']

    # creating input ids by tokenizing the question
    input_ids = tokenize_function(question, tokenizer)
    input_ids = {key: torch.tensor(array) for key, array in input_ids.items()}

    # generating the output
    outputs = model(**input_ids)
    logits = outputs.logits  # Shape: [batch_size, num_choices]
    print(logits)
    # Find all indices to have at least 80% of the max score
    # probabilities = F.softmax(logits, dim=1)  # Convert logits to probabilities
    # print(probabilities)
    # max_score = probabilities.max().item()  # Use max probability
    #### better approach: using min_score
    #min_score = logits.min().item()
    #threshold = 2 * min_score  # Compute threshold based on probabilities

    ### next approach: using a threshold from deviation
    mean_score = logits.mean().item()
    std_dev = logits.std().item()

    # Define a threshold based on deviation from the mean
    threshold = mean_score + (0.4 * std_dev)
    high_score_options = (logits >= threshold).nonzero(as_tuple=True)[1]  # Get the indices of valid options
    # high_score_options = (probabilities >= threshold).nonzero(as_tuple=True)[1]  # Get the indices of valid options

    # List the corresponding options
    high_score_answers = [options[idx] for idx in high_score_options.tolist()]
    intended_answer_binary = [1 if option in intended_answer else 0 for option in options]

    predicted_answer_binary = [1 if option in high_score_answers else 0 for option in options]

    if metric is not None:
        metric.add_batch(predictions=predicted_answer_binary, references=intended_answer_binary)

    return intended_answer, intended_answer_binary, predicted_answer_binary, high_score_answers



Text output

In [12]:
def text_model_output(question, pipeline):
    '''
    Handles an open text question and summarizes it
    parameter:
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the full context of the question as a string
    - summary: the generated summary as a string
    '''
    intended_answer = question['context']
    summary = pipeline(intended_answer, max_length=len(intended_answer), do_sample=False)
    return intended_answer, summary[0]['summary_text']



Phone Number output

In [13]:
def number_model_output(model, tokenizer, question, metric=None):
    '''
    Handles a question where the context should contain a phone number and generates an answer to that question
    '''
    intended_answer = question['intended_answer'][0]

    input_ids = tokenize_function(question, tokenizer)
    output = model(**input_ids)
    start_logits, end_logits = output.start_logits, output.end_logits

    # Get most probable start and end index
    start_idx = torch.argmax(start_logits, dim=1).item()
    end_idx = torch.argmax(end_logits, dim=1).item() + 1  # Include last token

    # Convert token IDs to text
    predicted_tokens = input_ids["input_ids"][0][start_idx:end_idx]
    predicted_number = tokenizer.decode(predicted_tokens, skip_special_tokens=True)

    if metric is not None:
        metric.add(predictions=predicted_number, references=intended_answer)

    return intended_answer, predicted_number



Date output

In [14]:
def convert_date_format(date_str):
  try:
    parsed_date = parser.parse(date_str)
    return parsed_date.strftime('%Y-%m-%d')
  except Exception as e:
    return date_str

def find_date_and_convert(input_string):
  date_regex = r'\b(?:\d{1,2}(?:st|nd|rd|th)?\s+[A-Za-z]+\s+\d{4}|\d{1,2}[./-]\d{1,2}[./-]\d{2,4}|\b[A-Za-z]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})\b'
  match = re.search(date_regex, input_string)
  if match:
    extracted_date = match.group(0)
    formatted_date = convert_date_format(extracted_date)
    return formatted_date
  else:
    return input_string




In [15]:
def date_model_output(model, tokenizer, question, metric=None):
    '''
    Handles a question where the context should contain a date and generates an answer to that question
    '''
    intended_answer = question['intended_answer'][0]

    input_ids = tokenize_function(question, tokenizer)
    output = model(**input_ids)
    start_logits, end_logits = output.start_logits, output.end_logits
    # Get most probable start and end index
    start_idx = torch.argmax(start_logits, dim=1).item()
    end_idx = torch.argmax(end_logits, dim=1).item() + 1  # Include last token

    # Convert token IDs to text
    predicted_tokens = input_ids["input_ids"][0][start_idx:end_idx]
    predicted_answer = tokenizer.decode(predicted_tokens, skip_special_tokens=True)
    formatted_predicted_answer = find_date_and_convert(predicted_answer)

    if metric is not None:
        metric.add(predictions=formatted_predicted_answer, references=intended_answer)

    return intended_answer, formatted_predicted_answer



## Model Selection

In [17]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
exact_match = evaluate.load("exact_match")

In [16]:
summarization_pipeline = pipeline("summarization", model="t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [23]:
model_name = "bert-base-cased"
model = AutoModelForMultipleChoice.from_pretrained(model_name, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [18]:
oe_model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased-distilled-squad")
oe_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

We look how accurate the model is for the open-ended questions. Is there a need to look for other models?

We test everything on the train dataset (Doesn't matter, because we do not fine-tune at this stage)

In [26]:
oe_qa_train_dataset = pd.DataFrame(qa_dataset['train'].filter(lambda example: example['type'] in ['DATE', 'NUMBER']))
oe_qa_train_dataset.shape

Filter:   0%|          | 0/1104 [00:00<?, ? examples/s]

(86, 8)

In [27]:
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(model, tokenizer, oe_model, oe_tokenizer, oe_qa_train_dataset, mc_metric=clf_metrics, oe_metric=exact_match)
print(f"The exact_match metric for all open-ended questions in the train dataset: {oe_metric_result['exact_match']}")

======= Wrong answer =======
Question: When do you wish to receive a follow-up?
Context: How about we touch base again on January 15th?  That works for me.
The intended answer was: 2025-01-15
The predicted answer was: january 15th

======= Wrong answer =======
Question: When do you wish to receive a follow-up?
Context: How about we connect again on January 17th?  That works for me.
The intended answer was: 2025-01-17
The predicted answer was: january 17th

======= Wrong answer =======
Question: When do you wish to receive a follow-up?
Context: How about we follow up around January 22nd of 2025? That should work nicely.
The intended answer was: 2025-01-22
The predicted answer was: 22nd of 2025

The exact_match metric for all open-ended questions in the train dataset: 0.9651162790697675


Is there a possibility to write a map function? Insert it here!!

No need of searching for another model here, we can concentrate on the mc questions

In [28]:
mc_train_qa_dataset = pd.DataFrame(qa_dataset['train'].filter(lambda example: example['type'] in ['MULTI_SELECT', 'SINGLE_SELECT']))
mc_train_qa_dataset.shape

model_results = []

Filter:   0%|          | 0/1104 [00:00<?, ? examples/s]

##### BERT base model (cased)
Trained from scratch

In [25]:
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(model, tokenizer, oe_model, oe_tokenizer, mc_train_qa_dataset, mc_metric=clf_metrics, oe_metric=exact_match)

Streaming output truncated to the last 5000 lines.
The intended answer was: ['High-speed interconnect testing']
The predicted answer was: ['Automotive radar target simulation', 'Noise figure measurements']
The intended answer in BINARY was: [0, 0, 0, 0, 1]
The predicted answer in BINARY was: [1, 1, 0, 0, 0]

tensor([[-0.7140, -0.7189, -0.7236, -0.7183, -0.7252, -0.7269, -0.7186, -0.7242,
         -0.7153, -0.7232, -0.7220, -0.7312, -0.7133]],
       grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Who to copy in follow up
Context: Hmm, I think I should copy Stephan Maier, Joachim Wagner, Oliver Eibel, Sandro Kalter and Tim Persson. Those seem like the people I'm supposed to follow up with.
The intended answer was: ['Stephan Maier', 'Joachim Wagner', 'Oliver Eibel', 'Sandro Kalter', 'Tim Persson']
The predicted answer was: ['Stephan Maier', 'Joachim Wagner', 'Oliver Eibel', 'Johannes Wagner', 'Sandro Kalter', 'Tim Persson']
The intended answer in BINARY was: [1, 1, 0, 1, 

In [ ]:
print(f"The metrics for all mc questions in the train dataset:\n{model_name}: {mc_metric_result}")
mc_metric_result['model_name'] = model_name
model_results.append(mc_metric_result)

##### XLNet base model (cased)




In [39]:
model_name = "xlnet/xlnet-base-cased"
mc_model = XLNetForMultipleChoice.from_pretrained(model_name)
mc_tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(mc_model, mc_tokenizer, oe_model, oe_tokenizer, mc_train_qa_dataset, mc_metric=clf_metrics, oe_metric=exact_match)
print(f"The metrics for all open-ended questions in the train dataset:\n{mc_metric_result}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
The predicted answer was: ['MY-SYSTEM', 'JS EcoLine']
The intended answer in BINARY was: [1, 0, 1, 1, 0, 0]
The predicted answer in BINARY was: [1, 0, 0, 1, 0, 0]

tensor([[0.1831, 0.1568]], grad_fn=<ViewBackward0>)
tensor([[ 0.2677,  0.1753,  0.1103, -0.0263]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: What kind of follow up is planned
Context: Okay, for follow up, I can either call you, *phone*, or we can *schedule a visit*. If neither is needed, we'll take *no action*.
The intended answer was: ['Phone', 'Schedule a Visit', 'No action']
The predicted answer was: ['Email']
The intended answer in BINARY was: [0, 1, 1, 1]
The predicted answer in BINARY was: [1, 0, 0, 0]

tensor([[-0.0766, -0.0440,  0.0239,  0.0230,  0.1300,  0.0417,  0.1068]],
       grad_fn=<ViewBackward0>)
tensor([[ 0.2129, -0.2678,  0.2089,  0.0660,  0.1731,  0.2041]],
       grad_fn=<ViewBackward0>)
======= Wrong answer =======


In [40]:
print(f"The metrics for all mc questions in the train dataset:\n{model_name}: {mc_metric_result}")
mc_metric_result['model_name'] = model_name
model_results.append(mc_metric_result)

The metrics for all mc questions in the train dataset:
xlnet/xlnet-base-cased: {'accuracy': 0.7078977932636469, 'f1': 0.5054080629301868, 'precision': 0.5538793103448276, 'recall': 0.46473779385171793}


##### RoBERTa base model of FacebookAI
Roberta models are using the hyperparameters of underlying bert model -> less fine-tuning necessary

In [41]:
model_name = "FacebookAI/roberta-base"
mc_model = AutoModelForMultipleChoice.from_pretrained(model_name)
mc_tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(mc_model, mc_tokenizer, oe_model, oe_tokenizer, mc_train_qa_dataset, mc_metric=clf_metrics, oe_metric=exact_match)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
tensor([[0.1364, 0.1373, 0.1391, 0.1375, 0.1388, 0.1386, 0.1323]],
       grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Size of the trade fair team (on average)
Context: Hmm, on average, I think the trade fair team would be more than 40 people, so something around that number sounds right.
The intended answer was: more than 40
The predicted answer was: 11-15
The intended answer in BINARY was: [0, 0, 0, 0, 0, 0, 1]
The predicted answer in BINARY was: [0, 0, 1, 0, 0, 0, 0]

tensor([[0.1318, 0.1324, 0.1339, 0.1312, 0.1301, 0.1254, 0.1323, 0.1301, 0.1338,
         0.1336, 0.1323, 0.1329, 0.1320]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Who to copy in follow up
Context: I'd copy Stephan Maier, Joachim Wagner, Oliver Eibel, Johannes Wagner, Jessica Hanke, and Tim Persson;  they all need to be in the loop on this follow up.
The intended answer was: ['Stephan Maier', 'Joachim Wagner', 

In [45]:
print(f"The metrics for all mc questions in the train dataset:\n{model_name}: {mc_metric_result}")
mc_metric_result['model_name'] = model_name
model_results.append(mc_metric_result)

The metrics for all mc questions in the train dataset:
FacebookAI/roberta-base: {'accuracy': 0.575687185443283, 'f1': 0.27989487516425754, 'precision': 0.3075812274368231, 'recall': 0.25678119349005424}


#### ALBERT base
No pretraining before, training is made from scratch

In [46]:
model_name = "albert/albert-base-v2"
mc_model = AutoModelForMultipleChoice.from_pretrained(model_name)
mc_tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [47]:
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(mc_model, mc_tokenizer, oe_model, oe_tokenizer, mc_train_qa_dataset, mc_metric=clf_metrics, oe_metric=exact_match)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
======= Wrong answer =======
Question: Which language is wanted for communication? 
Context: I'd prefer Japanese, I guess.  I don't know what other languages were offered, but Japanese is what comes to mind.
The intended answer was: Japanese 
The predicted answer was: Italian
The intended answer in BINARY was: [0, 0, 1, 0, 0]
The predicted answer in BINARY was: [0, 1, 0, 0, 0]

tensor([[0.1016, 0.2867, 0.1963, 0.1779, 0.2825]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: What is the type of contact?
Context: Oh, um, I guess it could be a Supplier, or maybe a New customer or Prospect. It might even be Press or media. Could it also be a Competitor. I don't know, maybe it's any of
The intended answer was: ['Supplier', 'New customer / Prospect', 'Press / media', 'Competitor']
The predicted answer was: ['Supplier', 'Competitor']
The intended answer in BINARY was: [0, 1, 1, 1, 1]
The predicted answer in BI

In [48]:
print(f"The metrics for all mc questions in the train dataset:\n{model_name}: {mc_metric_result}")
mc_metric_result['model_name'] = model_name
model_results.append(mc_metric_result)

The metrics for all mc questions in the train dataset:
albert/albert-base-v2: {'accuracy': 0.6314363143631436, 'f1': 0.3789954337899543, 'precision': 0.4129353233830846, 'recall': 0.350210970464135}


### Fine-tuning a model

The XLNet model outperforms every of the other models, so we use that one


In [19]:
def preprocess_function(example):
    '''
    Converts the string input, which is a question with its context and the given options for multi-/single-select questions, into IDs the model later can make sense of. Distinguishes between multi-/single-select and the other questions
    parameters:
    - expample: question of the QA-dataset with all its entries (question, context, options, type are urgently necessary)
    - tokenizer: tokenizer of the model
    output:
    - tokenized: tokenized input example
    '''
    if example["type"] == "SINGLE_SELECT" or example["type"] == "MULTI_SELECT":
      number_of_options = len(example["options"])
      first_sentence = [[example["context"]] * number_of_options]  # Repeat context for each option
      second_sentence = [[example["question"] + " " + option] for option in example["options"]]  # Pair with each option
      tokenized = tokenizer(
          sum(first_sentence, []),
          sum(second_sentence, []),
          padding="longest",
          truncation=True
      )
      # Un-flatten
      return {k: [v[i:i+number_of_options] for i in range(0, len(v), number_of_options)] for k, v in tokenized.items()}

    elif example['type'] == 'NUMBER':
      tokenized = tokenizer(
          example['context'],
          example['question'],
          truncation="only_second",
          max_length=384,
          padding="max_length",
          return_tensors="pt"
      )
    else:
      tokenized = tokenizer(
          example['question'],
          example['context'],
          truncation="only_second",
          max_length=384,
          padding="max_length",
          return_tensors="pt"
      )

    return tokenized

In [20]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0] else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)

        # Determine the maximum number of choices in the batch
        max_choices = max(len(label) for label in labels)
        # Pad missing choices for each feature
        for feature in features:
            num_choices = len(feature["input_ids"])
            while len(feature["input_ids"]) < max_choices:
                for key in feature.keys():
                    feature[key].append([0] * len(feature[key][0]))  # Pad with zeros

        # Flatten for tokenization
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(max_choices)]
            for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Reshape tensors to match (batch_size, num_choices, sequence_length)
        batch = {k: v.view(batch_size, max_choices, -1) for k, v in batch.items()}

        # Handle label padding if necessary
        for i, label in enumerate(labels):
            if isinstance(label, list):  # Handle MULTI_SELECT cases
                labels[i] += [0] * (max_choices - len(label))  # Pad labels with 0s
            else:
                labels[i] = label  # Keep as-is for SINGLE_SELECT
        batch["labels"] = torch.tensor(labels, dtype=torch.float).view(batch_size, -1)

        return batch


In [21]:
metric = evaluate.load("f1")

In [22]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = []
    for i in range(len(logits)):
      mean_score = logits[i].mean().item()
      std_dev = logits[i].std().item()

      # Define a threshold based on deviation from the mean
      threshold = mean_score + (0.4 * std_dev)
      prediction = (logits[i] >= threshold).astype(int)
      metric.add_batch(predictions=prediction, references=labels[i].astype(int))
    return metric.compute(average="macro") # maybe with parameter average = "macro"

In [39]:
def fine_tune_model(dataset, tokenizer, model, epochs, output_dir):
    # Preprocess the dataset
    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    # Define training arguments
    training_args = TrainingArguments(output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        logging_dir="./logs",
        learning_rate=4e-5,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=10,
        load_best_model_at_end=True,
        report_to="none"
    )
    data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    # Train the model
    trainer.train()

    drive.mount('/content/drive')

    model_path = f"/content/{output_dir}"
    save_path = f"/content/drive/MyDrive/mc_models/{output_dir}"
    # Create the directory if it does not exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print(f"Directory created: {save_path}")
    else:
        print("Directory already exists!")
    !cp -r {model_path} {save_path}  # Copy model to Google Drive


In [41]:
model_name = "bert-base-cased"
bert_model = AutoModelForMultipleChoice.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Try to apply fine-tuning methods

In [29]:
mc_qa_dataset = qa_dataset.filter(lambda example: example['type'] in ['SINGLE_SELECT', 'MULTI_SELECT'])
mc_qa_dataset.shape

Filter:   0%|          | 0/1104 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

{'train': (937, 7), 'test': (235, 7)}

In [30]:
sample_mc_qa_dataset = mc_qa_dataset.filter(lambda example, idx: idx > 105 and idx < 120, with_indices=True)
sample_mc_qa_dataset.shape

Filter:   0%|          | 0/937 [00:00<?, ? examples/s]

Filter:   0%|          | 0/235 [00:00<?, ? examples/s]

{'train': (14, 7), 'test': (14, 7)}

In [ ]:
fine_tune_model(mc_qa_dataset, tokenizer, bert_model, 7, "bert_fine_tuned")

Map:   0%|          | 0/937 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

<ipython-input-39-25329a740e37>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [33]:
model_name = "albert/albert-base-v2"
albert_model = AutoModelForMultipleChoice.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
fine_tune_model(sample_mc_qa_dataset, tokenizer, albert_model, 7, "albert/albert-base-v2")

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

<ipython-input-24-319c72042967>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,2.969500,2.752164,0.285463
2,2.696700,2.852798,0.237407
3,2.698100,2.861069,0.214931
4,2.683800,2.859945,0.225911
5,2.657500,2.845096,0.241281
6,2.598900,2.842575,0.245650
7,2.600700,2.840811,0.245650


Mounted at /content/drive
cp: cannot create directory '/content/drive/MyDrive/albert/albert-base-v2': No such file or directory


In [16]:
def accuracy(answer_comparison):
    '''
    Computes the total accuracy and accuracy for each question type for the passed list of dicts. One dict in the list is one question with keys 'answer', 'predicted_answer', 'type'
    parameters:
    - list of dicts with entries 1) predicted answer 2) answer 3) type of question
    '''
    correct_multi_select = 0
    correct_single_select = 0
    correct_text = 0
    correct_number = 0
    correct_date = 0
    correct_total = 0
    total = 0

    for entry in answer_comparison:
        question_type = entry['type']
        if entry['intended_answer'] == entry['predicted_answer']:
            if question_type == 'MULTI_SELECT':
                correct_multi_select += 1
                total_multi_select += 1
            elif question_type == 'SINGLE_SELECT':
                correct_single_select += 1
                total_single_select += 1
            elif question_type == 'TEXT':
                correct_text += 1
                total_text += 1
            elif question_type == 'NUMBER':
                correct_number += 1
                total_number += 1
            elif question_type == 'DATE':
                correct_date += 1
                total_date += 1
            else:
              continue
            correct_total += 1
        total += 1
    accuracy_total = correct_total / total
    accuracy_multi_select = correct_multi_select / total_multi_select
    accuracy_single_select = correct_single_select / total_single_select
    accuracy_text = correct_text / total_text
    accuracy_number = correct_number / total_number
    accuracy_date = correct_date / total_date
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date
'''
print_out_model_quality: takes the computations of function accuracy() and prints them out
parameters:
- accuracy_total
- accuracy_multi_select
- accuracy_single_select
- accuracy_text
- accuracy_number
- accuracy_date
'''
def print_out_model_quality(accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date):
    # accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date = accuracy(model, tokenizer, questions)
    print(f"""Accuracy values of model: {model.name_or_path}\n
    Total: {accuracy_total}\n
    Multi-select: {accuracy_multi_select}\n
    Single-select: {accuracy_single_select}\n
    Text: {accuracy_text}\n
    Number: {accuracy_number}\n
    Date: {accuracy_date}\n""")
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date




## Testing / Evaluation

In [197]:
# Example dataset

url = "https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/datasets/test_qa_dataset_with_answers.json"
data = pd.read_json(url)
# Convert to DataFrame for easy handling
test_df = pd.DataFrame(data)

# Map the intended answer to the index of the option
test_df['label'] = test_df.apply(lambda x: np.array([1 if option in x['intended_answer'] else 0 for option in x['options']]) if x['type'] in ['SINGLE_SELECT', 'MULTI_SELECT'] else np.array([0]), axis=1)

test_dataset = Dataset.from_pandas(df)

In [200]:
df_test_dataset = pd.DataFrame(test_dataset)
mc_results, oe_results, mc_metric_result, oe_metric_result = model_output(model, tokenizer, oe_model, oe_tokenizer, df_test_dataset, sum_pipeline=summarization_pipeline, mc_metric=clf_metrics, oe_metric=exact_match)

tensor([[-0.2242, -0.1725, -0.1572]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: When does the contact person wish to receive a follow up?
Context: Okay, so for the follow up, I think either **2 weeks** or **3 weeks** would work; they seem to be the options I have to choose from.
The intended answer was: ['2 weeks', '3 weeks']
The predicted answer was: ['3 weeks']
The intended answer in BINARY was: [0, 1, 1]
The predicted answer in BINARY was: [0, 0, 1]

tensor([[-0.5023, -0.4445, -0.4900]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: When does the contact person wish to receive a follow up?
Context: I'd prefer a follow up in either **1 week**, or **2 weeks**, whichever is more convenient for you.
The intended answer was: ['1 week', '2 weeks']
The predicted answer was: ['2 weeks']
The intended answer in BINARY was: [1, 1, 0]
The predicted answer in BINARY was: [0, 1, 0]

tensor([[0.4463, 0.5764, 0.5914, 0.5636, 0.5752]], grad_fn=<ViewBackward0>)

Your max_length is set to 100, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


tensor([[0.6425, 0.1219, 0.4493, 0.6006, 0.5004]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Searches a solution for
Context: I'm looking for something to help me, maybe to scan business cards or capture trade fair contacts, those sound helpful.
The intended answer was: ['Scan business cards', 'Capture trade fair contacts']
The predicted answer was: ['Scan business cards', 'Improve CRM data quality']
The intended answer in BINARY was: [1, 0, 0, 0, 1]
The predicted answer in BINARY was: [1, 0, 0, 1, 0]

======= Wrong answer =======
Question: Any additional notes?
Context: No, I don't think I have any additional notes at this time, but thank you for asking!
The intended answer was: No, I don't think I have any additional notes at this time, but thank you for asking!
The predicted answer was: no, I don't think I have any additional notes at this time . thank you for asking . if you have any questions, please contact me .

tensor([[0.6722, 0.7328, 0.2587, 0.7632, 0.82

Your max_length is set to 100, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


tensor([[0.4228, 0.4920, 0.2957, 0.4459]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: What kind of follow up is planned
Context: Okay, let me see... for follow up, we could send an **email**, or, it's also possible that we take **no action**. I guess those are the options.
The intended answer was: ['Email', 'No action']
The predicted answer was: ['Phone']
The intended answer in BINARY was: [1, 0, 0, 1]
The predicted answer in BINARY was: [0, 1, 0, 0]

======= Wrong answer =======
Question: Notes
Context: Actually, I don't think I have anything else to add right now. Thanks for asking, though.
The intended answer was: Actually, I don't think I have anything else to add right now. Thanks for asking, though.
The predicted answer was: i don't think I have anything else to add right now . thanks for asking, though, though . i'm not sure what to add .

tensor([[-0.0370,  0.3140,  0.6275,  0.4256,  0.5646,  0.0793,  0.6133,  0.4504]],
       grad_fn=<ViewBackward0>)
=====

...

tensor([[-0.4211, -0.4193, -0.4385, -0.3406, -0.2801, -0.3940]],
       grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: What is the contact person interested in?
Context: I'm not sure, but maybe they're interested in additive manufacturing, automation, assembly systems, or joining systems for large components.  Or something else entirely.
The intended answer was: ['100 Additive Manufacturing', '200 Automation', '234 Assembly Systems', '256 Joining Systems for large components', 'Others']
The predicted answer was: ['234 Assembly Systems', '256 Joining Systems for large components']
The intended answer in BINARY was: [1, 1, 0, 1, 1, 1]
The predicted answer in BINARY was: [0, 0, 0, 1, 1, 0]

======= Wrong answer =======
Question: Any additional notes?
Context: No, I think that covers everything.  I don't have any other notes to add at this time.
The intended answer was: No, I think that covers everything.  I don't have any other notes to add at this time.
The predicted ans

Your max_length is set to 100, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


tensor([[-0.0111, -0.1080, -0.1695]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Notes
Context: No, I don't believe I have anything else to add at this time.  Everything relevant is already included.
The intended answer was: No, I don't believe I have anything else to add at this time.  Everything relevant is already included.
The predicted answer was: no, I don't believe I have anything else to add at this time . everything relevant is already included . if you have any questions, please contact me .

tensor([[ 0.3995,  0.0526,  0.5249,  0.5096,  0.4494,  0.4474,  0.4722,  0.0683,
          0.1341, -0.1039]], grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: What industry are you operating in?
Context: I'm in network operations and infrastructure.  That's what I do; I handle the networks and their underlying systems.
The intended answer was: Network Operators & Infrastructure
The predicted answer was: Government
The intended answer in BINARY was: [0,

Your max_length is set to 100, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


======= Wrong answer =======
Question: Any additional notes?
Context: No, I don't think so; that covers everything.
The intended answer was: No, I don't think so; that covers everything.
The predicted answer was: no, I don't think so; that covers everything . no, that's what I'm talking about . it's a bit of a mystery .

tensor([[ 0.6079, -0.1601,  0.2418,  0.2396,  0.5395]],
       grad_fn=<ViewBackward0>)
======= Wrong answer =======
Question: Searches a solution for
Context: I'd try scanning business cards to get them into my system, then clean up my CRM to make sure everything's organized, and finally improve the CRM data quality so it's accurate.
The intended answer was: ['Scan business cards', 'Clean up CRM', 'Improve CRM data quality']
The predicted answer was: ['Scan business cards', 'Capture trade fair contacts']
The intended answer in BINARY was: [1, 1, 0, 1, 0]
The predicted answer in BINARY was: [1, 0, 0, 0, 1]

tensor([[-0.3179, -0.1102, -0.1583, -0.0837, -0.1268]],
      

In [202]:
with open('bert_mc_results.json', 'w') as fp:
    json.dump(mc_results, fp)